# Depuracion de datos mixtos

El presente notebook realiza la exploración inicial y depuración de datos agregados de vegetación consolidados por el SMBYC.
Se requieren los siguientes archivos:

1. `asignacion`: Tabla en formato csv. Contiene información básica de las parcelas (ubicación, custodio, etc.).
2. `individuos`: Tabla en formato csv. Contiene medidas y demás datos relacionos con los individuos (diámetro, altura, parcela, placa, etc.). 

In [3]:
import pandas as pd
import numpy as np
%matplotlib inline

# Asignar nombres de archivos a variables
asignacion = "../data/quimera/asignacion.csv"
individuos =  "../data/quimera/newind.csv"

In [43]:
asig = pd.read_csv(asignacion)
ind = pd.read_csv(individuos, low_memory=False) # low_memory asume que los datos estan homogeneamente tipificados.
tot = ind.join(asig.set_index('Plot'), on='Plot', rsuffix='_a') # concatenacion de ambas tablas usando en indice de parcela (`Plot`) como columna de referencia

### Variables para los nombres de las columnas

A continuación se adjudica una variable a cada nombre de la columna en ambos archivos. Si los nombres no son los mencionados deben ser actualizados en concordancia. El nombre de la columna que contiene el indice de la parcela debe ser igual en ambos archivos.

In [5]:
FID = 'FID' # Indice de medicion (int)
Plot = 'Plot' # Indice parcela (int)
D = 'D'# Diametro del tallo en cm (float)
H = 'H' # Altura total del tallo en m (float)
X = 'X' # Coordenada X en m (float)
Y = 'Y' # Coordenada Y en m (float)
Placa = 'Placa' # Placa de referencia del individuo (str, aunque la mayoria son int)
Densidad = 'Densidad' # Densidad de la madera en gr/ml (float)
Fuente_densidad = 'Fuente_densidad' # Referencia bibliografica de la densidad de la madera (str)
Habito = 'Habito' # Clasificacion de referencia (str: 'Arborea', 'Palma', 'No-Arborea', 'Exotica', 'Paramo', 'Mangle').
Entra_calculos = 'Entra_calculos' # Inclusion en analisis (str: 'Si', 'No')
Familia = 'Familia' # Familia taxonomica (str)
Autor_familia = 'Autor_familia' # Autor familia taxonomica (str)
Genero = 'Genero' # Genero taxonomica (str)
Autor_genero = 'Autor_genero' # Autor genero taxonomica (str)
Estado_epiteto = 'Estado_epiteto' # Incertidumbre de la determinacion especifica (str: 'aff.', 'cf.', 'vs.', 'gr.')
Epiteto = 'Epiteto' # Epiteto especifico, si indeterminado a especie contiene 'sp.' (str)
Autor_especie = 'Autor_especie' # Autor epiteto taxonomico (str)
Morfoespecie = 'Morfoespecie' # Concatenacion de los campos Genero y Epiteto (str).

fields_ind = [FID, Plot, D, H, X, Y, Placa, Densidad, Fuente_densidad, Habito, Entra_calculos, Familia, Autor_familia, Genero, Autor_genero, Estado_epiteto, Epiteto, Autor_especie, Morfoespecie]


In [94]:
100 * 100 * 0.02

200.0

In [110]:
cusdep = asig.groupby([Custodio_abreviado,Departamento]).size().reset_index()
print cusdep[cusdep[0] > 10]

    Custodio_abreviado     Departamento    0
0                  CAO          Caqueta   11
3                  CVC            Choco   77
4                  CVC  Valle del Cauca  860
5            Codechocó            Choco  257
6           Conciviles            Choco   15
7         Corantioquia        Antioquia  709
9          Corpochivor           Boyaca  275
11         Corponariño           Nariño   87
12       Corporinoquía           Arauca   22
13       Corporinoquía           Boyaca   12
14       Corporinoquía         Casanare   75
15       Corporinoquía     Cundinamarca   60
16       Corporinoquía          Vichada  116
19       Duivenvoorden         Amazonas   84
21               Duque         Amazonas   41
22               Duque          Caqueta   40
23             Exp-Ant        Antioquia   15
25                 FPR         Amazonas   12
26                 FPR          Caqueta   56
34                IAvH           Caldas   62
36                IAvH     Cundinamarca   11
39        

In [45]:
Area = 'Area' # Superficie de la parcela en hectareas (float64)
Year = 'Year' # Año de levantamiento de datos (int64)
Tipo_parcela = 'Tipo_parcela' # Clase de parcela (str: 'Temporal', 'Permanente')
Custodio = 'Custodio' # Autor o custodio de la informacion (str)
Custodio_abreviado = 'Custodio_abreviado' # Abreviatura del autor o custodio de la informacion (str)
Parcela_original = 'Parcela_original' # Codigo de la parcela empleado por el custodio (str)
Proyecto = 'Proyecto' # Codigo del proyecto bajo el cual la parcela fue establecida (str)
PID = 'PID' # Que carajos es esto???????????????
X = 'X' # Coordenada X WGS84 zona 18N en m, origen en 0°, -75° (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Y = 'Y' # Coordenada x WGS84 zona 18N en m, origen en 0°, -75° (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
X_MAGNA = 'X_MAGNA' # Coordenada x Magna en m (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Y_MAGNA = 'Y_MAGNA' # Coordenada y Magna en m (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Acceso = 'Acceso' # Clase de acceso permitido al IDEAM por el custodio (str: 'Confidencial','Público').
Departamento = 'Departamento' # Departamento str.
Municipio = 'Municipio' # Municipio str.
CAR = 'CAR' # Corporacion autonoma regional con jurisdiccion en el sitio de muestreo (str).
UAESPNN = 'UAESPNN' # Unidad del sistema de areas protegidas con jurisdiccion en el sitio de muestreo (str).
Region = 'Region' # Region geografica de Colombia (str: 'Amazonia', 'Caribe', 'Andes', 'Pacifico', 'Orinoquia', 'Andina').
Escenario_referencia = 'Escenario_referencia' # Unidad geografica de referencia (str: 'Amazonia', 'Noroccidental', 'Caribe', 'Suroccidental', 'Antioquia','Andes oriental', 'Eje cafetero', 'Nororiental', 'Orinoquia')
ECP = 'ECP' # ????????????????????????????????? (float64)
Holdridge = 'Holdridge' # Clasificacion climatica de Holdridge, modelo 2014 (str)
Provincia = 'Provincia' # Provincia bioclimatica, modelo 2014 (str: 'Wet forest', 'Moist forest', 'Dry forest')
Caldas_Lang = 'Caldas_Lang' # Clasificacion climatica Caldas-Lang, modelo 2014 (str)
Martonne = 'Martonne' # Clasificacion climatica Martonne, modelo 2014 (str: 'Bosque lluvioso','Bosque lluvioso estacional','Bosque húmedo','Bosque subhúmedo')
Eq1 = 'Eq1' # Valor de biomasa al aplicar la ecuacion 1 (int64)
Eq2 = 'Eq2' # Valor de biomasa al aplicar la ecuacion 2 (int64)
Eq3 = 'Eq3' # Valor de biomasa al aplicar la ecuacion 3 (int64)

fields_asig = [Plot, Area, Year, Tipo_parcela, Custodio, Custodio_abreviado, Parcela_original, Proyecto, PID, X, Y, X_MAGNA, Y_MAGNA, Acceso, Departamento, Municipio, CAR, UAESPNN, Region, Escenario_referencia, ECP, Holdridge, Provincia, Caldas_Lang, Martonne, Eq1, Eq2, Eq3]

### Verificar si las columnas tienen el tipo de dato apropiado

In [92]:
# Verificar si todas las columnas tienen el tipo de dato adecuado
for fi in [FID, Plot]:
    if ind[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, ind[fi].dtype)
        
for fi in [D, H, X, Y, Densidad]:
    if ind[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, ind[fi].dtype)
        
for fi in [Placa, Fuente_densidad, Habito, Familia, Autor_familia, Genero, Autor_genero, Estado_epiteto, Epiteto, Autor_especie, Morfoespecie]:
    non_strings = ind[fi].dropna()[~ind[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)

try:
    ind[Entra_calculos].replace(to_replace = ['Si','No'], value = [True, False], inplace = True)
except TypeError, ErrorMessage:
    if ErrorMessage.args[0] == "Cannot compare types 'ndarray(dtype=bool)' and 'str'":
        pass
    else:
        raise
except:
    raise

if ind[Entra_calculos].dtype != np.bool_:
    print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(Entra_calculos, ind[Entra_calculos].dtype)

# Verificacion de tipo de datos en tabla asignacion    

asig[X].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[Y].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[X_MAGNA].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[Y_MAGNA].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[[X,Y,X_MAGNA,Y_MAGNA]] = asig[[X,Y,X_MAGNA,Y_MAGNA]].apply(pd.to_numeric)
for fi in [Plot, Year, X, Y, X_MAGNA, Y_MAGNA, Eq1, Eq2, Eq3]:
    if asig[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, asig[fi].dtype)
        
for fi in [Area]:
    if asig[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de float64).".format(fi, asig[fi].dtype)
        
for fi in [Tipo_parcela, Custodio, Custodio_abreviado, Parcela_original, Departamento, Municipio, CAR, UAESPNN, Region, Escenario_referencia, Holdridge, Provincia, Caldas_Lang, Martonne]:
    non_strings = asig[fi].dropna()[~asig[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)

### Verificar si los valores estan en el rango apropiado

En este punto del proceso sólo se verifica que los valores tengan sentido lógico. Una mayor depuración de los datos se realizará después de que éstos sean incluidos en la base de datos.

In [7]:
# Indice no debe contener duplicado
if len(ind[ind[FID].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(individuos)
    
# Rango de diametro = 10-770 (diametro de General Sherman)
if ind[D].min() < 10:
    print "Existen valores de diametro inferiores a 10 cm"
if ind[D].max() > 770:
    print "Existen valores de diametro dudosamente altos"
    
# Rango de alturas = 1 - 100
if ind[H].min() < 1:
    print "Existen valores de altura inferiores a 1 m"
if ind[H].max() > 770:
    print "Existen valores de altura dudosamente altos"

# Rango de densidades de madera = 0.08 - 1.39 (Global Wood Density Database compilada por Chave y diponible en http://datadryad.org/repo/handle/10255/dryad.235)
if ind[Densidad].min() < 0.08:
    print "Existen valores de densidad inferiores a 0.8 gr/ml"
if ind[Densidad].max() > 1.39:
    print "Existen valores de densidad superiores a 1.39 gr/ml"
    
# Estados de incertidumbre taxonomica = 'aff.', 'cf.', 'vel sp. aff.'
for est in ind[Estado_epiteto].dropna()unique():
    if est not in ['aff.', 'cf.', 'vel sp. aff.']:
        print "{0} no es un estado de incertidumbre de determincacion aceptado".format(est)
        
# Indice de parcela no debe tener duplicados
if len(asig[asig[Plot].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(asignacion)
    
# Verificar areas, rango permitido: 0.02-25 ha
if asig[H].min() < 0.02:
    print "Algunos valores de area de parcela estan por debajo del valor minimo"
if asig[H].max() > 25:
    print "Algunos valores de area de parcela son sospechosamente altos"
    
# Rango de años permitido: 1990-2017
if asig[Year].min() < 1990:
    print "Realmente hay datos levantados antes de 1990?"
if asig[Year].max() > 2017:
    print "Parece que algunos datos provienen del futuro. Verificar fechas."

# Verificar correspondencia 1:1 entre el indice propio de parcela y el indice del custodio     
if filter(lambda x: x != 1, asig.groupby([Plot, Parcela_original]).size()):
    print "No hay concordancia entre las asignaciones de indices propios y externos:"
    multiPlots = asig.groupby([Plot, Parcela_original]).size().reset_index()
    print multiPlots[multiPlots[0] > 1]
    

In [41]:
sorted(list(ind.Estado_epiteto.dropna().unique()))

['aff.', 'cf.', 'gr.', 'vs.']

In [38]:
ind[(ind.Estado_epiteto == 'gr.') | (ind.Estado_epiteto == 'vs.')][[Familia,Genero,Epiteto,Estado_epiteto]]

,Familia,Genero,Epiteto,Estado_epiteto
4086,Moraceae,Brosimum,acutifolium,vs.
52465,Malvaceae,Matisia,ochrocalyx,gr.
52525,Malvaceae,Matisia,leptandra,gr.


In [42]:
asig.dtypes

Plot                      int64
Area                    float64
Year                      int64
Tipo_parcela             object
Custodio                 object
Custodio_abreviado       object
Parcela_original         object
Proyecto                 object
PID                       int64
X                        object
Y                        object
X_MAGNA                  object
Y_MAGNA                  object
Acceso                   object
Departamento             object
Municipio                object
CAR                      object
UAESPNN                  object
Region                   object
Escenario_referencia     object
ECP                     float64
Holdridge                object
Provincia                object
Caldas-Lang              object
Martonne                 object
Eq1                       int64
Eq2                       int64
Eq3                       int64
dtype: object

In [23]:
for d in range(10,450,10):
    heights = ind.H[(ind.D >= d) & (ind.D < (d+10))]
    if len(heights) > 20:
        mean = heights.mean()
        std = heights.std()
        print d, d+10
        print len(ind[(ind.D >= d) & (ind.D < (d+10)) & (ind.H < (mean - 2*std))])
        print tot[(tot.D >= d) & (tot.D < (d+10)) & (tot.H < (mean - 2*std))].groupby(['Custodio_abreviado']).size()
        print len(ind[(ind.D >= d) & (ind.D < (d+10)) & (ind.H > (mean + 2*std))])
        print tot[(tot.D >= d) & (tot.D < (d+10)) & (tot.H > (mean + 2*std))].groupby(['Custodio_abreviado']).size()
        #ind.H[(ind.D >= d) & (ind.D < (d+10)) & (ind.H < (mean + 2*std)) & (ind.H > (mean - 2*std))].hist(bins=20)
        print ""

10 20
1261
Custodio_abreviado
CVC              187
Codechocó        129
Corantioquia     473
Corpochivor        1
Corponariño       36
Cortolima          2
Duivenvoorden      3
Duque              2
Exp-Ant            6
FPR               35
IAvH              33
IDEAM             16
Natura            14
Sinchi            23
UNAL               2
USAID             18
Álvarez          281
dtype: int64
5800
Custodio_abreviado
CRC                 7
CVC               466
Codechocó         179
Corantioquia       85
Corpochivor         2
Corponariño       111
Corpourabá         17
Cortolima          76
Duivenvoorden     271
Duque             264
Exp-Ant           178
FPR               105
IAvH               82
IDEAM             418
Incoplan            7
Sinchi            528
Tropenbos          36
UNAL               42
USAID            1197
Álvarez          1729
dtype: int64

20 30
1405
Custodio_abreviado
CRC               1
CVC              16
Codechocó       324
Corantioquia    733
Corponariño 

In [ ]:
ind.st

In [ ]:
print chave(20, -70, -3)
print chave(25, -70, -3)
print chave(30, -70, -3)

In [ ]:
ind.plot.scatter(x=D, y=H)